In [2]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np 
import torch.optim as optim 
from torch.utils.data import Dataset,DataLoader
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
torch.manual_seed(42)

if torch.backends.mps.is_available():
    print('MPS is available bitch')
    device = torch.device("mps")
else:
    torch.device('cpu')

MPS is available bitch


In [5]:

training_data_load = Path("data/fashion-mnist_train.csv")
test_data_load = Path("data/fashion-mnist_test.csv")

#checking if they exist 

if training_data_load.exists() and test_data_load.exists():
    print("Data files found.")
else:
    print("Data files not found.")

Data files not found.


In [6]:
df1=pd.read_csv(training_data_load, on_bad_lines='skip')
df2=pd.read_csv(test_data_load, on_bad_lines='skip')

FileNotFoundError: [Errno 2] No such file or directory: 'data/fashion-mnist_train.csv'

In [7]:
main_df=pd.concat([df1,df2])
#x and y

X=main_df.iloc[:,1:]
Y=main_df.iloc[:,0]


X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
#scaling the data
X_train=X_train/255.0
X_test=X_test/255.0

NameError: name 'df1' is not defined

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        # Reshaping features to [N, 1, 28, 28] for image data in which 1 channel greyscale image of 28x28 pixels and -1 indicates the batch size placeholder 
        self.features=torch.tensor(features.to_numpy(),dtype=torch.float32).reshape(-1,1,28,28)
        self.labels=torch.tensor(labels.to_numpy(),dtype=torch.long)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index],self.labels[index]
    
training_data=CustomDataset(X_train,y_train)
test_data=CustomDataset(X_test,y_test)

In [ ]:
training_data_load=DataLoader(dataset=training_data,shuffle=True,batch_size=32,pin_memory=True)
test_data_load=DataLoader(dataset=test_data,shuffle=False,batch_size=32,pin_memory=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self,input_features):
        super().__init__()

        self.features=nn.Sequential(
            nn.Conv2d(in_channels=input_features,out_channels=32,kernel_size=3,padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),

            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )

        #Passing dummy tensor to get the size after conv layers
        with torch.no_grad():
            dummy_tensor=torch.zeros(1,input_features,28,28)
            dummy_output=self.features(dummy_tensor)
            flattened_size=dummy_output.numel()

        self.size=flattened_size

        self.labels=nn.Sequential(
            
            nn.Flatten(),
            nn.Linear(flattened_size,128),
            nn.ReLU(),

            nn.Dropout(p=0.3),
            nn.Linear(128,64),
            nn.ReLU(),

            nn.Linear(64,10)
            
        )

    def forward(self,features):

        x=self.features(features)
        x=self.labels(x)

        return x


In [ ]:
def train_model(epochs,training_data_load,model,optimizer,Loss,device):

    for epocs in range(1,epochs):

        total_loss=0

        for batch_Features,batch_labels in training_data_load:

            batch_Features,batch_labels=batch_Features.to(device),batch_labels.to(device)

            #forward_propaagartion
            prediction=model(batch_Features)

            #loss_calculation
            loss=Loss(prediction,batch_labels)

            #backpropagation and zero_gradients 
            optimizer.zero_grad()
            loss.backward()

            #optimizer
            optimizer.step()
            
            #add the loss of the batch to the total loss
            total_loss+=loss.item()

        # Calculate the average loss for the current epoch.
        avg_loss=total_loss/len(training_data_load)

        # Print the epoch number and the average training loss.
        print(f'epoch {epocs} and the avg loss {avg_loss:.2f}')


def evaluate_test_data(model,test_data_load,device):

    #putting model to eval mode
    model.eval()

    total=0
    correct=0

    with torch.inference_mode():

        for batch_features,batch_labels in test_data_load:
            
            batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

            #forward_propagation
            prediction=model(batch_features)

            #finding the maximum probability
            _,prediction=torch.max(prediction,1)

            #total size
            total+=batch_features.shape[0]

            #correct predictions
            correct+=(prediction==batch_labels).sum().item()

    accuracy=correct/total

    return accuracy

def evaluate_training_data(model,training_data_load,device):

    #set the model to device mode
    model.eval()
    
    #set 2 variables to keep the track of toal and correct predictions
    total=0
    correct=0


    with torch.inference_mode():

        for batch_features,batch_labels in training_data_load:

            #set them to device 
            batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)

            #forward_propagation
            prediction=model(batch_features)
            
            #categroy with the maximum probability 
            _,prediction=torch.max(prediction,1)
            
            #total_size
            total+=prediction.shape[0]

            #correct predictions
            correct+=(prediction==batch_labels).sum().item()

    accuracy=correct/total

    return accuracy